# Segmenting and Clustering Neighborhoods in Toronto
## Part 3 - Exploring Toronto


Start by importing the necessary packages

In [472]:
# import packages
import numpy as np
import pandas as pd

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

import geocoder

import requests
import json
from pandas.io.json import json_normalize

print("Packages imported")

Packages imported


The second notebook saved the pandas dataframe to a .csv file.
The first step it to retrieve the datarame from this file.
We also display the dataframe.

In [473]:
# import csv to dataframe
df_toronto=pd.read_csv('Toronto_neighborhoods_postalcodes.csv')

# display dataframe
df_toronto

,Unnamed: 0,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.752935,-79.335641
1,1,M4A,North York,Victoria Village,43.728102,-79.311890
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
4,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
...,...,...,...,...,...,...
98,98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653340,-79.509766
99,99,M4Y,Downtown Toronto,Church and Wellesley,43.666659,-79.381472
100,100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.648700,-79.385450
101,101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.632798,-79.493017


We can get some basic information on the data set

In [474]:
print('The dataframe has {} boroughs and {} postal districts.'.format(
        len(df_toronto['Borough'].unique()),
        df_toronto.shape[0]
    )
)

The dataframe has 10 boroughs and 103 postal districts.


In [475]:
# get the coordinates of Toronto

address = 'Toronto, Ontario'

g = geocoder.arcgis('Toronto, Ontario')

latitude = g.lat
longitude = g.lng
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))


The geographical coordinates of Toronto are 43.648690000000045, -79.38543999999996.


In [476]:
# Plot a map of Toronto with the postal districts
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)


# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

Now go ahead and start clustering with data from foursquare. We can begin by defining some useful functions that can be used to analyse the data returned.




In [477]:
# Define some useful functions

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# function tthat returns all the venues around a position
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# function to return the x most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now we define the credentials to access the foursquare API

In [478]:
# Define the credentials for foursquare

CLIENT_ID = '???' # your Foursquare ID
CLIENT_SECRET = '???' # your Foursquare Secret
VERSION = '20190529' # Foursquare API version

We will begin by exploring one of the neighbourhood, "Central Bay Street" 

In [479]:
# find the postal code for "Central Bay Street"

nh="Central Bay Street"

nh_lat=float(df_toronto[df_toronto['Neighborhood'].str.contains(nh)]['Latitude'])
nh_lng=float(df_toronto[df_toronto['Neighborhood'].str.contains(nh)]['Longitude'])

print("Neighborhood \"{}\" has a latitude of {} and a longitude of {}".format(nh, nh_lat, nh_lng))

Neighborhood "Central Bay Street" has a latitude of 43.65607218800005 and a longitude of -79.38565318999997


Now use the longitude and latitude to return information on that neighborhood

In [480]:
limit = 100 # limit for number of venues to be returned
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    nh_lat, 
    nh_lng, 
    radius, 
    limit)

url 

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed1818777af03001bb1fa51'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 54,
  'suggestedBounds': {'ne': {'lat': 43.660572192500055,
    'lng': -79.37944500279238},
   'sw': {'lat': 43.65157218350005, 'lng': -79.39186137720756}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e2284b11fc7c0ef9857d143',
       'name': 'Chatime 日出茶太',
       'location': {'address': '132 Dundas St W',
        'crossStreet': 'btwn Bay & University',
        'lat': 43.65554164147378,
        'lng': -79.38468427043244,
    

The json returned can be cleaned to yield a dataframe showing the venues present in the neighborhood

In [481]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Chatime 日出茶太,Bubble Tea Shop,43.655542,-79.384684
1,The Elm Tree Restaurant,Modern European Restaurant,43.657397,-79.383761
2,Textile Museum of Canada,Art Museum,43.654396,-79.386500
3,Rolltation,Japanese Restaurant,43.654918,-79.387424
4,Japango,Sushi Restaurant,43.655268,-79.385165


In [482]:
# How many venues are there

print('{} venues were returned by Foursquare for the neghborhood {}.'.format(nearby_venues.shape[0], nh))

if(nearby_venues.shape[0]==limit):
    print("Note: this is the maximum number requested by limit")

54 venues were returned by Foursquare for the neghborhood Central Bay Street.


In [483]:
# Use the function to get information on each neighborhood

toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                 latitudes=df_toronto['Latitude'],
                                  longitudes=df_toronto['Longitude']
                                )


Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [484]:
# Determine the number of venues returned

print(toronto_venues.shape)
toronto_venues.head()

(2279, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.752935,-79.335641,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.752935,-79.335641,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.728102,-79.311890,Tim Hortons,43.725517,-79.313103,Coffee Shop
3,Victoria Village,43.728102,-79.311890,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.728102,-79.311890,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection


In [485]:
print("Foursquare has returned information on {} venues in Toronto" .format(toronto_venues.shape[0]))

Foursquare has returned information on 2279 venues in Toronto


We can group the result by neigborhood in order to see the number of venues returned for each.

In [486]:
# group the results by neighborhood to get the number of venues in each

toronto_venues.groupby('Neighborhood').count()[["Venue"]].rename(
                       columns={'Venue':'Number of venues returned'}).sort_values(
                       'Number of venues returned', ascending=False)

,Number of venues returned
Neighborhood,
"Garden District, Ryerson",100
"Commerce Court, Victoria Hotel",100
"First Canadian Place, Underground city",100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100
Stn A PO Boxes,100
...,...
"Rouge Hill, Port Union, Highland Creek",1
Humber Summit,1
"Wexford, Maryvale",1


In [487]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

toronto_categories = toronto_venues.groupby('Venue Category').count()[["Venue"]].rename(
                       columns={'Venue':'Number of venues'}).sort_values(
                       'Number of venues', ascending=False)

print("\nThe top 5 most numerous categories are")

toronto_categories.head()


There are 262 uniques categories.

The top 5 most numerous categories are


,Number of venues
Venue Category,
Coffee Shop,185
Café,94
Restaurant,77
Park,56
Hotel,50


We now apply one hot coding to the data in order to prepare for clustering

In [488]:
# one hot encoding

toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# Drop the one hot coded Neighborhood column
toronto_onehot.drop('Neighborhood', axis=1, inplace=True)

# add neighborhood column back to dataframe
toronto_onehot.insert(0, 'Neighborhood', toronto_venues['Neighborhood'])

toronto_onehot




,Neighborhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2274,"Mimico NW, The Queensway West, South of Bloor,...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2275,"Mimico NW, The Queensway West, South of Bloor,...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2276,"Mimico NW, The Queensway West, South of Bloor,...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2277,"Mimico NW, The Queensway West, South of Bloor,...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [489]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped



,Neighborhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.1,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.047619,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
94,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
95,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.076923,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
96,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [490]:
toronto_grouped.shape

(98, 262)

We can show the top most commone venues in each neigborhood

In [491]:
# Top 5 most common venues for each neighborhood

num_top_venues = 5

for nh_val in toronto_grouped['Neighborhood']:
    print("----"+nh_val+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == nh_val].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
              venue  freq
0    Breakfast Spot   0.2
1   Badminton Court   0.2
2       Supermarket   0.2
3      Skating Rink   0.2
4  Sushi Restaurant   0.2


----Alderwood, Long Branch----
               venue  freq
0           Pharmacy   0.1
1        Coffee Shop   0.1
2  Convenience Store   0.1
3     Sandwich Place   0.1
4                Pub   0.1


----Bathurst Manor, Wilson Heights, Downsview North----
              venue  freq
0       Coffee Shop  0.10
1              Bank  0.10
2       Pizza Place  0.05
3     Shopping Mall  0.05
4  Sushi Restaurant  0.05


----Bayview Village----
                        venue  freq
0  Construction & Landscaping   0.5
1                       Trail   0.5
2               Movie Theater   0.0
3                   Nightclub   0.0
4                Night Market   0.0


----Bedford Park, Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.10
1    Sushi Restaurant  0.10
2         Coffee Shop  0.10
3  Italian Restaura

In [492]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']


for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Breakfast Spot,Supermarket,Badminton Court,Sushi Restaurant,Skating Rink
1,"Alderwood, Long Branch",Pizza Place,Pub,Dance Studio,Coffee Shop,Gas Station
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Pet Store,Deli / Bodega,Bridal Shop
3,Bayview Village,Construction & Landscaping,Trail,Yoga Studio,Farmers Market,Electronics Store
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Sushi Restaurant,Coffee Shop,Italian Restaurant,Indian Restaurant


Now we use kmeans clustering to define clusters of neighborhoods using the grouped one hot coded and grouped data

In [493]:
# set number of clusters
kclusters = 8

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 6, 1, 1, 0, 1, 1, 1])

In [494]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()


,Unnamed: 0,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,0,M3A,North York,Parkwoods,43.752935,-79.335641,2.0,Food & Drink Shop,Park,Farm,Eastern European Restaurant,Electronics Store
1,1,M4A,North York,Victoria Village,43.728102,-79.311890,1.0,Pizza Place,Financial or Legal Service,Portuguese Restaurant,French Restaurant,Coffee Shop
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,1.0,Pub,Café,Athletics & Sports,Bakery,Theater
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211,1.0,Clothing Store,Cosmetics Shop,Toy / Game Store,Restaurant,Bookstore
4,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390,1.0,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Sushi Restaurant


We can now use folium to plot the clusters on the map of toronto

In [495]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    if np.isnan(cluster)==False:
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster)-1],
            fill=True,
            fill_color=rainbow[int(cluster)-1],
            fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

And finally, examine the neighborhoods and their venues in each cluster.

In [496]:
for i in range(0,kclusters):
    print("Cluster {}\n===========" .format(i))
    display(toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]])

Cluster 0
Cluster 1
Cluster 2
Cluster 3
Cluster 4
Cluster 5
Cluster 6
Cluster 7


,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,M4B,-79.311529,0.0,Pizza Place,Fast Food Restaurant,Pharmacy,Pet Store,Gym / Fitness Center
10,M6B,-79.447500,0.0,Pizza Place,Mediterranean Restaurant,Gas Station,Pub,Japanese Restaurant
11,M9B,-79.554089,0.0,Pizza Place,Sandwich Place,Gift Shop,Tea Room,Chinese Restaurant
14,M4C,-79.308507,0.0,Grocery Store,Breakfast Spot,Fast Food Restaurant,Bar,Sushi Restaurant
18,M1E,-79.185284,0.0,Restaurant,Fast Food Restaurant,Pizza Place,Coffee Shop,Breakfast Spot
32,M1J,-79.228725,0.0,Sandwich Place,Fast Food Restaurant,Big Box Store,Indian Restaurant,Fish & Chips Shop
40,M3K,-79.467631,0.0,Grocery Store,Shopping Mall,Pizza Place,Coffee Shop,Vietnamese Restaurant
46,M3L,-79.512027,0.0,Grocery Store,Shopping Mall,Pizza Place,Coffee Shop,Vietnamese Restaurant
47,M4L,-79.315578,0.0,Fast Food Restaurant,Pizza Place,Sandwich Place,Italian Restaurant,Burrito Place
51,M1M,-79.228353,0.0,Sandwich Place,Pharmacy,Liquor Store,Bistro,Coffee Shop


,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,M4A,-79.311890,1.0,Pizza Place,Financial or Legal Service,Portuguese Restaurant,French Restaurant,Coffee Shop
2,M5A,-79.353041,1.0,Pub,Café,Athletics & Sports,Bakery,Theater
3,M6A,-79.451211,1.0,Clothing Store,Cosmetics Shop,Toy / Game Store,Restaurant,Bookstore
4,M7A,-79.389390,1.0,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Sushi Restaurant
7,M3B,-79.357220,1.0,Trail,Athletics & Sports,Spa,Other Great Outdoors,Bank
...,...,...,...,...,...,...,...,...
96,M4X,-79.367326,1.0,Coffee Shop,Pizza Place,Italian Restaurant,Pub,Restaurant
97,M5X,-79.381884,1.0,Coffee Shop,Café,Hotel,Japanese Restaurant,Gym
99,M4Y,-79.381472,1.0,Coffee Shop,Japanese Restaurant,Restaurant,Gay Bar,Café
100,M7Y,-79.385450,1.0,Coffee Shop,Hotel,Restaurant,Japanese Restaurant,Café


,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,-79.335641,2.0,Food & Drink Shop,Park,Farm,Eastern European Restaurant,Electronics Store
5,M9A,-79.528953,2.0,Park,Baseball Field,Skating Rink,Yoga Studio,Electronics Store
27,M2H,-79.356566,2.0,Residential Building (Apartment / Condo),Park,Dog Run,Yoga Studio,Eastern European Restaurant
49,M6L,-79.488724,2.0,Park,Bakery,Yoga Studio,Farm,Eastern European Restaurant
57,M9M,-79.544381,2.0,Playground,Park,Dog Run,Fish Market,Fish & Chips Shop
66,M2P,-79.398355,2.0,Park,Convenience Store,Bank,Yoga Studio,Electronics Store
68,M5P,-79.410280,2.0,Park,Yoga Studio,Donut Shop,Flower Shop,Fish Market
73,M4R,-79.406456,2.0,Playground,Garden,Park,Gym Pool,Falafel Restaurant
88,M8V,-79.499751,2.0,Park,Skating Rink,Grocery Store,Tennis Court,Yoga Studio
91,M4W,-79.376706,2.0,Playground,Grocery Store,Park,Candy Store,Field


,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
12,M1C,-79.157368,3.0,Bar,Yoga Studio,Eastern European Restaurant,Food,Flower Shop


,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
50,M9L,-79.557174,4.0,Sporting Goods Shop,Donut Shop,Food,Flower Shop,Fish Market


,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
35,M4J,-79.33389,5.0,Bus Stop,Yoga Studio,Food & Drink Shop,Flower Shop,Fish Market


,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
6,M1B,-79.189913,6.0,Trail,Yoga Studio,Farm,Eastern European Restaurant,Electronics Store
26,M1H,-79.238813,6.0,Gaming Cafe,Trail,Farmers Market,Electronics Store,Elementary School
39,M2K,-79.376921,6.0,Construction & Landscaping,Trail,Yoga Studio,Farmers Market,Electronics Store


,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
71,M1R,-79.300537,7.0,Auto Garage,Yoga Studio,Farmers Market,Electronics Store,Elementary School
